In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow-text==2.17.0


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow-text==2.17.0 (from versions: none)
ERROR: No matching distribution found for tensorflow-text==2.17.0


In [3]:
!pip install --upgrade tensorflow_hub


In [8]:
pip install tensorflow-text
import tensorflow as tf
print(tf.__version__)


SyntaxError: invalid syntax (1251662673.py, line 1)

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

ModuleNotFoundError: No module named 'tensorflow_text'

In [ ]:
import pandas as pd
df=pd.read_csv("spam (1).csv")
df.head(5)

In [ ]:
df.groupby('Category').describe()

In [ ]:
747/4825

In [ ]:
df_spam=df[df['Category']=='spam']
df_spam.shape

In [ ]:
df_ham=df[df['Category']=='ham']
df_ham.shape

In [ ]:
df_ham_down=df_ham.sample(df_spam.shape[0])
df_ham_down.shape

In [ ]:
df_balanced=pd.concat([df_ham_down,df_spam])
df_balanced.shape

In [ ]:
df_balanced['Category'].value_counts()

In [ ]:
df_balanced['spam']=df_balanced['Category'].apply(lambda x:1 if x=='spam' else 0)
df_balanced.sample(5)

In [ ]:

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['spam'], stratify=df_balanced['spam'])

In [ ]:

X_train.head(4)

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']


In [ ]:
get_sentence_embeding([
    "500$ discount. hurry up",
    "Bhavin, are you up for a volleybal game tomorrow?"]
)


In [ ]:
e = get_sentence_embeding([
    "banana",
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

In [ ]:
cosine_similarity([e[0]],[e[4]])

In [ ]:
cosine_similarity([e[3]],[e[4]])

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [ ]:
nt= tf.keras.layers.Dropout(0.1,name="dropout")(outputs['pooled_output'])
nt = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(nt)

In [ ]:
model = tf.keras.Model(inputs=[text_input], outputs = [nt])
model.summary()

In [ ]:

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              )

In [ ]:
model.fit(X_train, y_train, epochs=1)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [ ]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
cm

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_predicted))

In [ ]:
reviews = [
    'Enter a chance to win $5000, hurry up, offer valid until march 31, 2021',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
    "Bypoll results in 7 states: INDIA wins 2 seats, leading in 9; NDA ahead in 2"
]
model.predict(reviews)